In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
import gc


In [ ]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Training variables
BATCH_SIZE = 32
EPOCHS = 150
LEARNING_RATE = 0.00005

In [ ]:
# Transformations for the train, validation, and test datasets
train_transform = transforms.Compose([
    transforms.Resize(256, antialias=True),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.Resize((256, 256), antialias=True),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the train, validation, and test datasets
train_dataset = datasets.Flowers102(root="./data", split="train", transform=train_transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_dataset = datasets.Flowers102(root="./data", split="val", transform=val_test_transform, download=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_dataset = datasets.Flowers102(root="./data", split="test", transform=val_test_transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

In [ ]:
# Clear CUDA cache
torch.cuda.empty_cache()

# Create an instance of the CNN and move it to the device
cnn = CNN().to(device)

criterion = nn.CrossEntropyLoss()
# Set weight decay
WEIGHT_DECAY = 0.03

# Create the optimizer and add weight decay
optimizer = optim.Adam(cnn.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.1, verbose=True)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(16),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 224 -> 112
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(32),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 112 -> 56
        self.conv3 = nn.Sequential(nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 56 -> 28
        self.conv4 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(128),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 28 -> 14
        self.conv5 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(256),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 14 -> 7
        self.conv6 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(512),
                                   nn.ReLU(),
                                   nn.MaxPool2d(2, 2)) # 7 -> 3
        self.conv7 = nn.Sequential(nn.Conv2d(512, 1024, kernel_size=3, stride=1, padding=1),
                                   nn.BatchNorm2d(1024),
                                   nn.MaxPool2d(2, 2)) # 3 -> 1

        self.fc1 = nn.Sequential(nn.Linear(1024, 512),
                                 nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(512, 256),
                                 nn.ReLU())

        self.fc3 = nn.Sequential(nn.Linear(256, 102))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = x.view(-1, 1024)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x


In [1]:
# Train the CNN
for epoch in range(EPOCHS):
    cnn.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        if (i + 1) % 16 == 0:
            print(f"Epoch [{epoch + 1}/{EPOCHS}], Step [{i + 1}/{len(train_loader)}], Step Loss: {loss.item():.4f}")

    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total
    print(f'Epoch [{epoch+1}/{EPOCHS}], Training Loss: {train_loss:.4f}, Training Accuracy: {train_accuracy:.2f}%')

    # Evaluate model after each training epoch
    cnn.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = cnn(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader)
    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{EPOCHS}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')

    # Update the learning rate scheduler
    scheduler.step(val_loss)

# Test the CNN on the test set
cnn.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = cnn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')



100%|██████████| 344862509/344862509 [00:24<00:00, 13875396.99it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 561627.26it/s]


100%|██████████| 14989/14989 [00:00<00:00, 15461982.95it/s]


Epoch [1/160], Step [16/32], Step Loss: 4.5858
Epoch [1/160], Step [32/32], Step Loss: 4.5697
Epoch [1/160], Training Loss: 4.5824, Training Accuracy: 1.86%
Epoch [1/160], Validation Loss: 4.5647, Validation Accuracy: 2.25%
Epoch [2/160], Step [16/32], Step Loss: 4.3865
Epoch [2/160], Step [32/32], Step Loss: 4.3287
Epoch [2/160], Training Loss: 4.4093, Training Accuracy: 4.61%
Epoch [2/160], Validation Loss: 4.3262, Validation Accuracy: 6.47%
Epoch [3/160], Step [16/32], Step Loss: 4.2363
Epoch [3/160], Step [32/32], Step Loss: 4.2385
Epoch [3/160], Training Loss: 4.2157, Training Accuracy: 8.53%
Epoch [3/160], Validation Loss: 4.1437, Validation Accuracy: 9.90%
Epoch [4/160], Step [16/32], Step Loss: 3.9889
Epoch [4/160], Step [32/32], Step Loss: 3.9687
Epoch [4/160], Training Loss: 4.0005, Training Accuracy: 12.06%
Epoch [4/160], Validation Loss: 3.9344, Validation Accuracy: 12.35%
Epoch [5/160], Step [16/32], Step Loss: 3.8648
Epoch [5/160], Step [32/32], Step Loss: 3.5534
Epoch [5